![DLI Header](images/DLI_Header.png)

# Text Classification #

## Joint Intent and Slot classification ## 

Intent and slot classification is a task of classifying an Intent and detecting all relevant Slots (Entities) for this Intent in a query. For example, in the query:  `What is the weather in Santa Clara tomorrow morning?`, we would like to classify the query as a `weather` Intent, and detect `Santa Clara` as a `location` slot and `tomorrow morning` as a `date_time` slot. Intents and Slots names are usually task specific and defined as labels in the training data. This is a fundamental step that is executed in any task-driven Conversational Assistant.

NeMo's BERT-based model implementation enables to train and then detect both of these tasks together.

**Table of Contents**<br>
This notebook covers the below sections: 
* Dataset
    * Download and Preprocess data
    * Labeling Data (OPTIONAL)
* Model Training
    * Download Model
    * Make Predictions
* Evaluation

In [1]:
BRANCH = 'main'

In [2]:
from nemo.collections import nlp as nemo_nlp
from nemo.utils import logging

import os
import wget
import torch
import pytorch_lightning as pl
from omegaconf import OmegaConf

NOTE! Installing ujson may make loading annotations faster.


## Dataset and NeMo data format ##

In this tutorial we are going to use a virtual assistant interaction data set that can be downloaded from here: https://github.com/xliuhw/NLU-Evaluation-Data.
There are about 10K training and 1K testing queries which cover 64 various Intents and 55 Slots. 

To work with NeMo NLP classification model, this dataset should be first converted to the NeMo format, which requires next files:
- **dict.intents.csv** - list of all intent names in the data. One line per an intent name.
- **dict.slots.csv** - list of all slot names in the data. One line per a slot name. It is possible to use both: B- I- notations, for separating between first and intermediate tokens for multi token slots. Or just use one slot type for each token of multi token slot. Our recommendation is to use later one, since it is simpler and there is no visible degradation in performance.
- **train.tsv/test.tsv** - contain original queries, one per line, and intent number separated by tab. For example: `what alarms do i have set right now	0`. Intent numbers are according to the intent line in the intent dictionary file (dict.intents.csv) starting from 0. First line of these files contains a header line: `sentence \tab label`.
- **train_slot.tvs/test_slot.tsv** - contain one line per a query, where instead each token there is a number of the token from the slots dictionary file (dict.slots.csv), starting from 0. Last 'out-of scope' token is usually located in the last line of the dictionary. Example: `54 0 0 54 54 12 12` (numbers separated by space). No header line in these files.

We have provided the `import_dataset.py` and `assistant_utils.py` scripts, which convert the data into the NeMo data format for the Intent and Slot classification model. If you have your own annotated dataset in a different format, you will need to write a data converter. Possible recommended format for your own annotation, is to have one text file per all examples of one intent. With one line per query in a form like: `did i set an alarm to [alarm_type : wake up] in the [timeofday : morning]`, using brackets to define slot names. This is very similar to the assistant format from this example and you can use its converter to NeMo format with small changes. 

You can run this utility as follows:
```
python examples/nlp/intent_slot_classification/data/import_datasets.py --dataset_name=assistant --source_data_dir=source_dir_name --target_data_dir=target_dir_name
```

#### Preprocess Data ####

In [3]:
# you can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
DATA_DIR=os.path.join(DATA_DIR, 'NLU-Evaluation-Data')

In [4]:
# # download and unzip the example dataset from github
TEST_DIR='.'
print('Downloading dataset...')
wget.download('https://github.com/xliuhw/NLU-Evaluation-Data/archive/master.zip', TEST_DIR)
!unzip {TEST_DIR}/NLU-Evaluation-Data-master.zip -d {TEST_DIR}

Archive:  ./NLU-Evaluation-Data-master.zip
f6071b496b17d71e6eb43f543af0707f4ff30557
   creating: ./NLU-Evaluation-Data-master/
   creating: ./NLU-Evaluation-Data-master/AnnotatedData/
  inflating: ./NLU-Evaluation-Data-master/AnnotatedData/NLU-Data-Home-Domain-Annotated-All.csv  
   creating: ./NLU-Evaluation-Data-master/AnnotationGuidelines/
  inflating: ./NLU-Evaluation-Data-master/AnnotationGuidelines/NLU-MLtraining-AnnotationSchma.docx  
   creating: ./NLU-Evaluation-Data-master/Collected-Original-Data/
  inflating: ./NLU-Evaluation-Data-master/Collected-Original-Data/paraphrases_and_intents_26k_normalised_all.csv  
   creating: ./NLU-Evaluation-Data-master/CrossValidation/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/
   creating: ./NLU-Evaluation-Data-master/CrossValidation/autoGeneFromRealAnno/autoGene_2018_03_22-13_01_25_169/CrossVal

In [5]:
# convert the dataset to the NeMo format
!python import_datasets.py --dataset_name=assistant --source_data_dir={DATA_DIR} --target_data_dir={DATA_DIR}/nemo_format

NOTE! Installing ujson may make loading annotations faster.
[NeMo I 2024-08-23 17:52:56 assistant_utils:141] Processing assistant commands dataset and store at data/NLU-Evaluation-Data/nemo_format
[NeMo I 2024-08-23 17:52:56 assistant_utils:36] Copying files to input folder: data/NLU-Evaluation-Data/dataset
[NeMo I 2024-08-23 17:52:56 assistant_utils:53] Found 64 intents
[NeMo I 2024-08-23 17:52:56 assistant_utils:92] Found 55 slot types


## Data exploration
You can see the dataset in both the original and NeMo's formats. We have here 65 different Intents and 55 Slots, which could be typical commands for virtual assistants. Out of scope slot has the name 'O' and is the last in the dictionary of Slots. And we can see examples of queries and also format of training intent and slot files. 

In [6]:
# list of queries divided by intent files in the original training dataset
! ls -l {DATA_DIR}/dataset/trainset

total 1488
-rw-r--r-- 1 root root 22459 Mar  9  2022 alarm_query.csv
-rw-r--r-- 1 root root 13045 Mar  9  2022 alarm_remove.csv
-rw-r--r-- 1 root root 23552 Mar  9  2022 alarm_set.csv
-rw-r--r-- 1 root root  7911 Mar  9  2022 audio_volume_down.csv
-rw-r--r-- 1 root root 14525 Mar  9  2022 audio_volume_mute.csv
-rw-r--r-- 1 root root 14732 Mar  9  2022 audio_volume_up.csv
-rw-r--r-- 1 root root 27361 Mar  9  2022 calendar_query.csv
-rw-r--r-- 1 root root 27068 Mar  9  2022 calendar_remove.csv
-rw-r--r-- 1 root root 34492 Mar  9  2022 calendar_set.csv
-rw-r--r-- 1 root root 25138 Mar  9  2022 cooking_recipe.csv
-rw-r--r-- 1 root root 16357 Mar  9  2022 datetime_convert.csv
-rw-r--r-- 1 root root 23226 Mar  9  2022 datetime_query.csv
-rw-r--r-- 1 root root 12869 Mar  9  2022 email_addcontact.csv
-rw-r--r-- 1 root root 23510 Mar  9  2022 email_query.csv
-rw-r--r-- 1 root root 26902 Mar  9  2022 email_querycontact.csv
-rw-r--r-- 1 root root 30069 Mar  9  2022 email_sendemail.csv
-rw-r--r-- 

In [7]:
# print all intents from the NeMo format intent dictionary
!echo 'Intents: ' $(wc -l < {DATA_DIR}/nemo_format/dict.intents.csv)
!cat {DATA_DIR}/nemo_format/dict.intents.csv

Intents:  64
alarm_query
alarm_remove
alarm_set
audio_volume_down
audio_volume_mute
audio_volume_up
calendar_query
calendar_remove
calendar_set
cooking_recipe
datetime_convert
datetime_query
email_addcontact
email_query
email_querycontact
email_sendemail
general_affirm
general_commandstop
general_confirm
general_dontcare
general_explain
general_joke
general_negate
general_praise
general_quirky
general_repeat
iot_cleaning
iot_coffee
iot_hue_lightchange
iot_hue_lightdim
iot_hue_lightoff
iot_hue_lighton
iot_hue_lightup
iot_wemo_off
iot_wemo_on
lists_createoradd
lists_query
lists_remove
music_likeness
music_query
music_settings
news_query
play_audiobook
play_game
play_music
play_podcasts
play_radio
qa_currency
qa_definition
qa_factoid
qa_maths
qa_stock
recommendation_events
recommendation_locations
recommendation_movies
social_post
social_query
takeaway_order
takeaway_query
transport_query
transport_taxi
transport_ticket
transport_traffic
weather_query


In [8]:
# print all slots from the NeMo format slot dictionary
!echo 'Slots: ' $(wc -l < {DATA_DIR}/nemo_format/dict.slots.csv)
!cat {DATA_DIR}/nemo_format/dict.slots.csv

Slots:  55
alarm_type
app_name
artist_name
audiobook_author
audiobook_name
business_name
business_type
change_amount
coffee_type
color_type
cooking_type
currency_name
date
definition_word
device_type
drink_type
email_address
email_folder
event_name
food_type
game_name
game_type
general_frequency
house_place
ingredient
joke_type
list_name
meal_type
media_type
movie_name
movie_type
music_album
music_descriptor
music_genre
news_topic
order_type
person
personal_info
place_name
player_setting
playlist_name
podcast_descriptor
podcast_name
radio_name
relation
song_name
time
time_zone
timeofday
transport_agency
transport_descriptor
transport_name
transport_type
weather_descriptor
O


In [9]:
# examples from the intent training file
!head -n 10 {DATA_DIR}/nemo_format/train.tsv

sentence	label
what alarms do i have set right now	0
checkout today alarm of meeting	0
report alarm settings	0
see see for me the alarms that you have set tomorrow morning	0
is there an alarm for ten am	0
confirm the alarm time	0
show my alarms	0
at what time have you set alarm for me	0
please list active alarms	0


In [10]:
# examples from the slot training file
!head -n 10 {DATA_DIR}/nemo_format/train_slots.tsv

54 54 54 54 54 54 54 54
54 12 54 54 54
54 54 54
54 54 54 54 54 54 54 54 54 54 12 48
54 54 54 54 54 46 46
54 54 54 54
54 54 54
54 54 54 54 54 54 54 54 54
54 54 54 54
54 54 54 54 54 54


# Training model

## Model configuration

Our Joint Intent and Slot classification model is comprised of the pretrained [BERT](https://arxiv.org/pdf/1810.04805.pdf) model with an Intent and Slot Classification layer on top of it.

All model and training parameters are defined in the **intent_slot_classification_config.yaml** config file. This file is located in the folder **examples/nlp/intent_slot_classification/conf/**. It contains 2 main sections:
- **model**: All arguments that are related to the Model - language model, token classifier, optimizer and schedulers, datasets and any other related information

- **trainer**: Any argument to be passed to PyTorch Lightning

We will download the config file from repository for the purpose of the tutorial. If you have a version of NeMo installed locally, you can use it from the above folder.

In [11]:
WORK_DIR='WORK_DIR'
MODEL_CONFIG = "intent_slot_classification_config.yaml"
config_dir = WORK_DIR + '/configs/'
os.makedirs(config_dir, exist_ok=True)
if not os.path.exists(config_dir + MODEL_CONFIG):
    print('Downloading config file...')
    wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/intent_slot_classification/conf/' + MODEL_CONFIG, config_dir)
else:
    print ('config file already exists')

In [12]:
config_file = "intent_slot_classification_config.yaml"
config=OmegaConf.load(config_dir + "/" + config_file)

# print the entire configuration file
print(OmegaConf.to_yaml(config))

trainer:
  devices: 1
  num_nodes: 1
  max_epochs: 50
  max_steps: -1
  accumulate_grad_batches: 1
  precision: 32
  accelerator: gpu
  strategy: ddp
  log_every_n_steps: 1
  val_check_interval: 1.0
  enable_checkpointing: false
  logger: false
model:
  nemo_path: null
  data_dir: ???
  class_labels:
    intent_labels_file: intent_labels.csv
    slot_labels_file: slot_labels.csv
  class_balancing: null
  intent_loss_weight: 0.6
  pad_label: -1
  ignore_extra_tokens: false
  ignore_start_end: true
  train_ds:
    prefix: train
    batch_size: 32
    shuffle: true
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
  validation_ds:
    prefix: test
    batch_size: 32
    shuffle: false
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
  test_ds:
    prefix: test
    batch_size: 32
    shuffle: false
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
  tokenizer:
    tokenizer_name: ${model.langua

## Setting up Data within the config

Among other things, the config file contains dictionaries called train_ds and validation_ds. These are configurations used to setup the Dataset and DataLoaders of the corresponding config.

The converter utility creates both training and evaluation files in the same directory, so we need to specify `model.data_dir` parameter to this directory. Also notice that some config lines, including `model.data_dir`, have `???` in place of paths, this means that values for these fields are required to be specified by the user.

`config.model.intent_loss_weight` parameter - is a balance of training loss between Intent and Slot losses, a number between 0 to 1. Its default value is 0.6 which gives slightly higher priority to the Intent loss and it empirically works quite well. You can experiment with this value if you like.
Also you can try to change `config.model.class_balancing` parameter to `weighted_loss` and see if you get better accuracy.

Let's now add the data directory path to the config.

In [13]:
config.model.data_dir = f'{DATA_DIR}/nemo_format'

## Building the PyTorch Lightning Trainer

NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem. `config.trainer.max_epochs` - param defines number of training epochs. Usually 50-100 epochs or less should be enough to train on your data. Let's instantiate the Trainer object.

In [14]:
# lets modify some trainer configs
# checks if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.devices = 1
config.trainer.accelerator = accelerator

config.trainer.precision = 16 if torch.cuda.is_available() else 32

# for mixed precision training, uncomment the line below (precision should be set to 16 and amp_level to O1):
# config.trainer.amp_level = O1

# remove distributed training flags
config.trainer.strategy = 'auto'

# setup a small number of epochs for demonstration purposes of this tutorial
config.trainer.max_epochs = 5

trainer = pl.Trainer(**config.trainer)

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


## Initializing the model and Training

Initial statistics of the dataset will be displayed at the beginning of the training and then Intent and Slot classification report will be displayed after each training epoch.

In [15]:
# initialize the model
model = nemo_nlp.models.IntentSlotClassificationModel(config.model, trainer=trainer)

# train
trainer.fit(model)

[NeMo I 2024-08-23 17:53:00 intent_slot_classification_descriptor:87]  Stats calculating for train mode...
[NeMo I 2024-08-23 17:53:00 intent_slot_classification_descriptor:112] Three most popular intents in train mode:
[NeMo I 2024-08-23 17:53:00 data_preprocessing:194] label: 0, 175 out of 9960 (1.76%).
[NeMo I 2024-08-23 17:53:00 data_preprocessing:194] label: 2, 175 out of 9960 (1.76%).
[NeMo I 2024-08-23 17:53:00 data_preprocessing:194] label: 6, 175 out of 9960 (1.76%).
[NeMo I 2024-08-23 17:53:00 intent_slot_classification_descriptor:118] Three most popular slots in train mode:
[NeMo I 2024-08-23 17:53:00 data_preprocessing:194] label: 54, 53689 out of 65658 (81.77%).
[NeMo I 2024-08-23 17:53:00 data_preprocessing:194] label: 38, 1290 out of 65658 (1.96%).
[NeMo I 2024-08-23 17:53:00 data_preprocessing:194] label: 12, 1233 out of 65658 (1.88%).
[NeMo I 2024-08-23 17:53:00 intent_slot_classification_descriptor:121] Total Number of Intents: 9960
[NeMo I 2024-08-23 17:53:00 intent_

Using eos_token, but it is not set yet.
Using bos_token, but it is not set yet.
[NeMo W 2024-08-23 17:53:00 modelPT:244] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2024-08-23 17:53:04 intent_slot_classification_dataset:92] Setting max length to: 30
[NeMo I 2024-08-23 17:53:04 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2024-08-23 17:53:04 data_preprocessing:406] Min: 3 |                  Max: 30 |                  Mean: 9.241967871485944 |                  Median: 9.0
[NeMo I 2024-08-23 17:53:04 data_preprocessing:412] 75 percentile: 11.00
[NeMo I 2024-08-23 17:53:04 data_preprocessing:413] 99 percentile: 19.00
[NeMo I 2024-08-23 17:53:04 intent_slot_classification_dataset:121] 0 are longer than 30
[NeMo I 2024-08-23 17:53:05 intent_slot_classification_dataset:92] Setting max length to: 26
[NeMo I 2024-08-23 17:53:05 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2024-08-23 17:53:05 data_preprocessing:406] Min: 3 |                  Max: 26 |                  Mean: 9.342007434944238 |                  Median: 9.0
[NeMo I 2024-08-23 17:53:05 data_preprocessing:412] 75 percentile: 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[NeMo I 2024-08-23 17:53:07 modelPT:721] Optimizer config = Adam (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 2e-05
        maximize: False
        weight_decay: 0.01
    )
[NeMo I 2024-08-23 17:53:07 lr_scheduler:910] Scheduler "<nemo.core.optim.lr_scheduler.WarmupAnnealing object at 0x7fde6a83a3b0>" 
    will be used during training (effective maximum steps = -1) - 
    Parameters : 
    (last_epoch: -1
    max_steps: -1
    warmup_steps: null
    warmup_ratio: null
    )


...

Validation: 0it [00:00, ?it/s]

[NeMo I 2024-08-23 17:54:35 intent_slot_classification_model:267] Intent report: 
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                                0.00       0.00       0.00         19
    alarm_remove (label_id: 1)                               0.00       0.00       0.00         11
    alarm_set (label_id: 2)                                  0.00       0.00       0.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                          0.00       0.00       0.00         15
    audio_volume_up (label_id: 5)                            0.00       0.00       0.00         13
    calendar_query (label_id: 6)                             0.00       0.00       0.00         19
    calendar_remove (label_id: 7)                            0.00       0.00       0.00         19
    calendar_set (l

Validation: 0it [00:00, ?it/s]

[NeMo I 2024-08-23 17:54:57 intent_slot_classification_model:267] Intent report: 
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                                0.00       0.00       0.00         19
    alarm_remove (label_id: 1)                               0.00       0.00       0.00         11
    alarm_set (label_id: 2)                                  0.00       0.00       0.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                          0.00       0.00       0.00         15
    audio_volume_up (label_id: 5)                            0.00       0.00       0.00         13
    calendar_query (label_id: 6)                             0.00       0.00       0.00         19
    calendar_remove (label_id: 7)                            0.00       0.00       0.00         19
    calendar_set (l

`Trainer.fit` stopped: `max_epochs=5` reached.


After training for 5 epochs, which should take no more than few minutes, you can expect training precision for this data set to be around these numbers (the accuracy will gradually continue to improve for this dataset up to about 50 epochs of training): 
```
Intents:
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                               94.74      94.74      94.74         19
    alarm_remove (label_id: 1)                             100.00     100.00     100.00         11
    alarm_set (label_id: 2)                                 85.71      94.74      90.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                        100.00      86.67      92.86         15
    audio_volume_up (label_id: 5)                           56.52     100.00      72.22         13
    calendar_query (label_id: 6)                            55.00      57.89      56.41         19
    calendar_remove (label_id: 7)                           88.89      84.21      86.49         19
    calendar_set (label_id: 8)                              81.25      68.42      74.29         19
    cooking_recipe (label_id: 9)                            86.36     100.00      92.68         19
    datetime_convert (label_id: 10)                          0.00       0.00       0.00          8
    datetime_query (label_id: 11)                           65.52     100.00      79.17         19
    email_addcontact (label_id: 12)                        100.00      12.50      22.22          8
    email_query (label_id: 13)                              83.33      78.95      81.08         19
    email_querycontact (label_id: 14)                       62.50      78.95      69.77         19
    email_sendemail (label_id: 15)                          70.83      89.47      79.07         19
    general_affirm (label_id: 16)                           95.00     100.00      97.44         19
    general_commandstop (label_id: 17)                     100.00     100.00     100.00         19
    general_confirm (label_id: 18)                         100.00     100.00     100.00         19
    general_dontcare (label_id: 19)                        100.00     100.00     100.00         19
    general_explain (label_id: 20)                         100.00      94.74      97.30         19
    general_joke (label_id: 21)                            100.00     100.00     100.00         12
    general_negate (label_id: 22)                           95.00     100.00      97.44         19
    general_praise (label_id: 23)                          100.00      94.74      97.30         19
    general_quirky (label_id: 24)                           40.00      10.53      16.67         19
    general_repeat (label_id: 25)                          100.00     100.00     100.00         19
    iot_cleaning (label_id: 26)                             84.21     100.00      91.43         16
    iot_coffee (label_id: 27)                               94.74      94.74      94.74         19
    iot_hue_lightchange (label_id: 28)                      94.44      89.47      91.89         19
    iot_hue_lightdim (label_id: 29)                        100.00      83.33      90.91         12
    iot_hue_lightoff (label_id: 30)                         89.47      89.47      89.47         19
    iot_hue_lighton (label_id: 31)                           0.00       0.00       0.00          3
    iot_hue_lightup (label_id: 32)                          81.25      92.86      86.67         14
    iot_wemo_off (label_id: 33)                             60.00     100.00      75.00          9
    iot_wemo_on (label_id: 34)                             100.00      14.29      25.00          7
    lists_createoradd (label_id: 35)                        78.95      78.95      78.95         19
    lists_query (label_id: 36)                              78.95      78.95      78.95         19
    lists_remove (label_id: 37)                             90.00      94.74      92.31         19
    music_likeness (label_id: 38)                           70.59      66.67      68.57         18
    music_query (label_id: 39)                              77.78      73.68      75.68         19
    music_settings (label_id: 40)                            0.00       0.00       0.00          7
    news_query (label_id: 41)                               77.78      73.68      75.68         19
    play_audiobook (label_id: 42)                           90.00      94.74      92.31         19
    play_game (label_id: 43)                                80.00      84.21      82.05         19
    play_music (label_id: 44)                               53.85      73.68      62.22         19
    play_podcasts (label_id: 45)                            89.47      89.47      89.47         19
    play_radio (label_id: 46)                               93.75      78.95      85.71         19
    qa_currency (label_id: 47)                              95.00     100.00      97.44         19
    qa_definition (label_id: 48)                            85.00      89.47      87.18         19
    qa_factoid (label_id: 49)                               45.16      73.68      56.00         19
    qa_maths (label_id: 50)                                100.00     100.00     100.00         14
    qa_stock (label_id: 51)                                 95.00     100.00      97.44         19
    recommendation_events (label_id: 52)                    94.44      89.47      91.89         19
    recommendation_locations (label_id: 53)                 94.74      94.74      94.74         19
    recommendation_movies (label_id: 54)                   100.00     100.00     100.00         10
    social_post (label_id: 55)                              90.00      94.74      92.31         19
    social_query (label_id: 56)                             94.74     100.00      97.30         18
    takeaway_order (label_id: 57)                           93.75      78.95      85.71         19
    takeaway_query (label_id: 58)                           85.71      94.74      90.00         19
    transport_query (label_id: 59)                          83.33      78.95      81.08         19
    transport_taxi (label_id: 60)                          100.00     100.00     100.00         18
    transport_ticket (label_id: 61)                         89.47      89.47      89.47         19
    transport_traffic (label_id: 62)                       100.00     100.00     100.00         19
    weather_query (label_id: 63)                           100.00      89.47      94.44         19
    -------------------
    micro avg                                               85.04      85.04      85.04       1076
    macro avg                                               81.13      80.81      79.36       1076
    weighted avg                                            84.10      85.04      83.54       1076
    
Slots:
    label                                                precision    recall       f1           support   
    alarm_type (label_id: 0)                                 0.00       0.00       0.00          0
    app_name (label_id: 1)                                   0.00       0.00       0.00          6
    artist_name (label_id: 2)                                0.00       0.00       0.00         21
    audiobook_author (label_id: 3)                           0.00       0.00       0.00          1
    audiobook_name (label_id: 4)                             0.00       0.00       0.00         18
    business_name (label_id: 5)                             60.00      56.60      58.25         53
    business_type (label_id: 6)                              0.00       0.00       0.00         24
    change_amount (label_id: 7)                              0.00       0.00       0.00         25
    coffee_type (label_id: 8)                                0.00       0.00       0.00          4
    color_type (label_id: 9)                                 0.00       0.00       0.00         12
    cooking_type (label_id: 10)                              0.00       0.00       0.00          0
    currency_name (label_id: 11)                            84.09      75.51      79.57         49
    date (label_id: 12)                                     57.95      91.07      70.83        112
    definition_word (label_id: 13)                           0.00       0.00       0.00         20
    device_type (label_id: 14)                              74.55      51.25      60.74         80
    drink_type (label_id: 15)                                0.00       0.00       0.00          0
    email_address (label_id: 16)                             0.00       0.00       0.00         14
    email_folder (label_id: 17)                              0.00       0.00       0.00          1
    event_name (label_id: 18)                              100.00      13.24      23.38         68
    food_type (label_id: 19)                                51.72      69.77      59.41         43
    game_name (label_id: 20)                                60.00      14.29      23.08         21
    game_type (label_id: 21)                                 0.00       0.00       0.00          0
    general_frequency (label_id: 22)                         0.00       0.00       0.00          9
    house_place (label_id: 23)                              93.33      42.42      58.33         33
    ingredient (label_id: 24)                                0.00       0.00       0.00          6
    joke_type (label_id: 25)                                 0.00       0.00       0.00          4
    list_name (label_id: 26)                                 0.00       0.00       0.00         21
    meal_type (label_id: 27)                                 0.00       0.00       0.00          0
    media_type (label_id: 28)                                0.00       0.00       0.00         37
    movie_name (label_id: 29)                                0.00       0.00       0.00          0
    movie_type (label_id: 30)                                0.00       0.00       0.00          0
    music_album (label_id: 31)                               0.00       0.00       0.00          0
    music_descriptor (label_id: 32)                          0.00       0.00       0.00          3
    music_genre (label_id: 33)                               0.00       0.00       0.00          9
    news_topic (label_id: 34)                                0.00       0.00       0.00         17
    order_type (label_id: 35)                                0.00       0.00       0.00         17
    person (label_id: 36)                                   44.86      92.31      60.38         52
    personal_info (label_id: 37)                             0.00       0.00       0.00         20
    place_name (label_id: 38)                               71.25      77.03      74.03        148
    player_setting (label_id: 39)                            0.00       0.00       0.00          1
    playlist_name (label_id: 40)                             0.00       0.00       0.00          1
    podcast_descriptor (label_id: 41)                        0.00       0.00       0.00         13
    podcast_name (label_id: 42)                              0.00       0.00       0.00          4
    radio_name (label_id: 43)                               66.67      10.53      18.18         38
    relation (label_id: 44)                                  0.00       0.00       0.00         17
    song_name (label_id: 45)                                 0.00       0.00       0.00         22
    time (label_id: 46)                                     70.27      78.20      74.02        133
    time_zone (label_id: 47)                                 0.00       0.00       0.00          9
    timeofday (label_id: 48)                                 0.00       0.00       0.00         28
    transport_agency (label_id: 49)                          0.00       0.00       0.00          9
    transport_descriptor (label_id: 50)                      0.00       0.00       0.00          0
    transport_name (label_id: 51)                            0.00       0.00       0.00          4
    transport_type (label_id: 52)                           78.38      82.86      80.56         35
    weather_descriptor (label_id: 53)                        0.00       0.00       0.00         17
    O (label_id: 54)                                        92.42      98.80      95.50       5920
    -------------------
    micro avg                                               89.10      89.10      89.10       7199
    macro avg                                               21.86      18.56      18.18       7199
    weighted avg                                            84.42      89.10      86.01       7199
```

## Evaluation

In [16]:
# we will setup testing data reusing the same config (test section)
model.setup_test_data(test_data_config=config.model.test_ds)

# run the evaluation on the test dataset
trainer.test(model=model, verbose=False)

[NeMo I 2024-08-23 17:54:58 intent_slot_classification_dataset:92] Setting max length to: 26
[NeMo I 2024-08-23 17:54:58 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2024-08-23 17:54:58 data_preprocessing:406] Min: 3 |                  Max: 26 |                  Mean: 9.342007434944238 |                  Median: 9.0
[NeMo I 2024-08-23 17:54:58 data_preprocessing:412] 75 percentile: 11.00
[NeMo I 2024-08-23 17:54:58 data_preprocessing:413] 99 percentile: 19.00
[NeMo I 2024-08-23 17:54:58 intent_slot_classification_dataset:121] 0 are longer than 26


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
[NeMo W 2024-08-23 17:54:58 nemo_logging:349] /usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    


Testing: 0it [00:00, ?it/s]

[NeMo I 2024-08-23 17:55:00 intent_slot_classification_model:267] Intent report: 
    label                                                precision    recall       f1           support   
    alarm_query (label_id: 0)                                0.00       0.00       0.00         19
    alarm_remove (label_id: 1)                               0.00       0.00       0.00         11
    alarm_set (label_id: 2)                                  0.00       0.00       0.00         19
    audio_volume_down (label_id: 3)                          0.00       0.00       0.00          8
    audio_volume_mute (label_id: 4)                          0.00       0.00       0.00         15
    audio_volume_up (label_id: 5)                            0.00       0.00       0.00         13
    calendar_query (label_id: 6)                             0.00       0.00       0.00         19
    calendar_remove (label_id: 7)                            0.00       0.00       0.00         19
    calendar_set (l

[{'val_loss': 4.053339004516602,
  'intent_precision': 1.3011152744293213,
  'intent_recall': 1.3011152744293213,
  'intent_f1': 1.301114797592163,
  'slot_precision': 15.40491771697998,
  'slot_recall': 15.40491771697998,
  'slot_f1': 15.404916763305664}]

## Inference from Examples
Next step to see how the trained model will classify Intents and Slots for given queries from this domain. To improve the predictions you may need to train the model for more than 5 epochs.


In [17]:
queries = [
    'set alarm for seven thirty am',
    'lower volume by fifty percent',
    'what is my schedule for tomorrow',
]

# pred_intents, pred_slots = eval_model.predict_from_examples(queries, config.model.test_ds)
pred_intents, pred_slots = model.predict_from_examples(queries, config.model.test_ds)

logging.info('The prediction results of some sample queries with the trained model:')
for query, intent, slots in zip(queries, pred_intents, pred_slots):
    logging.info(f'Query : {query}')
    logging.info(f'Predicted Intent: {intent}')
    logging.info(f'Predicted Slots: {slots}')

[NeMo I 2024-08-23 17:55:01 intent_slot_classification_dataset:92] Setting max length to: 8
[NeMo I 2024-08-23 17:55:01 data_preprocessing:404] Some stats of the lengths of the sequences:
[NeMo I 2024-08-23 17:55:01 data_preprocessing:406] Min: 7 |                  Max: 8 |                  Mean: 7.666666666666667 |                  Median: 8.0
[NeMo I 2024-08-23 17:55:01 data_preprocessing:412] 75 percentile: 8.00
[NeMo I 2024-08-23 17:55:01 data_preprocessing:413] 99 percentile: 8.00
[NeMo I 2024-08-23 17:55:01 intent_slot_classification_dataset:121] 0 are longer than 8
[NeMo I 2024-08-23 17:55:01 3172433480:10] The prediction results of some sample queries with the trained model:
[NeMo I 2024-08-23 17:55:01 3172433480:12] Query : set alarm for seven thirty am
[NeMo I 2024-08-23 17:55:01 3172433480:13] Predicted Intent: email_query
[NeMo I 2024-08-23 17:55:01 3172433480:14] Predicted Slots: audiobook_name O audiobook_name O O movie_type
[NeMo I 2024-08-23 17:55:01 3172433480:12] Quer

![DLI Header](images/DLI_Header.png)